# Borrower reliability analysis
Project Description
The customer is the credit department of the bank. It is necessary to find out whether the marital status and the number of children of the client affect the fact of repaying the loan on time. Input data from the bank - statistics on the solvency of customers.

The results of the study will be taken into account when building a credit scoring model - a special system that assesses the ability of a potential borrower to return a loan to a bank.

Answer the questions

Is there a relationship between having children and repaying a loan on time?

Is there a relationship between marital status and loan repayment on time?

Is there a relationship between income level and loan repayment on time?

How do different purposes of a loan affect its repayment on time?

In [1]:
import pandas as pd

In [2]:
data=pd.read_csv('data.csv')

In [3]:
data.head()

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу


# Preprocessing data

In [4]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21525 non-null  int64  
 1   days_employed     19351 non-null  float64
 2   dob_years         21525 non-null  int64  
 3   education         21525 non-null  object 
 4   education_id      21525 non-null  int64  
 5   family_status     21525 non-null  object 
 6   family_status_id  21525 non-null  int64  
 7   gender            21525 non-null  object 
 8   income_type       21525 non-null  object 
 9   debt              21525 non-null  int64  
 10  total_income      19351 non-null  float64
 11  purpose           21525 non-null  object 
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


Columns 'days_employed' and 'total_income' have null values. Probably those people haven't worked yet because of the age. Or they have been working unofficially. Going to change those values to 0 cause other values may increase credit score which is not right. 

In [5]:
data['days_employed'] = data['days_employed'].fillna(0)
data['total_income'] = data['total_income'].fillna(0)

In [6]:
#checking
data.isnull().sum()

children            0
days_employed       0
dob_years           0
education           0
education_id        0
family_status       0
family_status_id    0
gender              0
income_type         0
debt                0
total_income        0
purpose             0
dtype: int64

In [7]:
data['days_employed'].describe()

count     21525.000000
mean      56678.874622
std      134870.763085
min      -18388.949901
25%       -2518.168900
50%        -982.531720
75%           0.000000
max      401755.400475
Name: days_employed, dtype: float64

Column 'days_employed' has negative values. Going to change it to absolute values and transform it to years(integer). Deleting the days employed column 

In [8]:
data['days_employed'] = data['days_employed'].abs()
data['years_employed']=data['days_employed']/365
data['years_employed']=data['years_employed'].astype('int')

In [9]:
del data['days_employed']

Changing 'education' and 'family status' columns to low register so it would be easier to analyze

In [10]:
data['education']=data['education'].str.lower()
data['family_status']=data['family_status'].str.lower()

Changing 'total income' column to 'int' data type to make easier analysis

In [11]:
data['total_income']=data['total_income'].astype('int')

In [12]:
data.head()

,children,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,years_employed
0,1,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875,покупка жилья,23
1,1,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080,приобретение автомобиля,11
2,0,33,среднее,1,женат / замужем,0,M,сотрудник,0,145885,покупка жилья,15
3,3,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628,дополнительное образование,11
4,0,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616,сыграть свадьбу,932


In [13]:
data['children'].value_counts()

 0     14149
 1      4818
 2      2055
 3       330
 20       76
-1        47
 4        41
 5         9
Name: children, dtype: int64

Filling mistake is observed. Going to change '-1' to '1' and '20' to '2'

In [14]:
data['children'] = data['children'].replace(-1, 1)
data['children'] = data['children'].replace(20, 2)

In [15]:
data['children'].value_counts()

0    14149
1     4865
2     2131
3      330
4       41
5        9
Name: children, dtype: int64

Now data looks quite clean. There are unreal values in 'years_employed' column, but we are not going to use these values in analysis. So no need to deal with it

In [16]:
data.head(10)

,children,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,years_employed
0,1,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875,покупка жилья,23
1,1,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080,приобретение автомобиля,11
2,0,33,среднее,1,женат / замужем,0,M,сотрудник,0,145885,покупка жилья,15
3,3,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628,дополнительное образование,11
4,0,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616,сыграть свадьбу,932
5,0,27,высшее,0,гражданский брак,1,M,компаньон,0,255763,покупка жилья,2
6,0,43,высшее,0,женат / замужем,0,F,компаньон,0,240525,операции с жильем,7
7,0,50,среднее,1,женат / замужем,0,M,сотрудник,0,135823,образование,0
8,2,35,высшее,0,гражданский брак,1,F,сотрудник,0,95856,на проведение свадьбы,18
9,0,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425,покупка жилья для семьи,5


# Dropping Duplicates

In [17]:
data.duplicated().sum()

71

In [18]:
data=data.drop_duplicates().reset_index(drop=True)

In [19]:
data.duplicated().sum()

0

# Lemmatization of 'purpose' column

In [20]:
from pymystem3 import Mystem
m=Mystem()

In [21]:
purposes = data['purpose'].unique().tolist()

In [22]:
lemmas='; '.join(purposes)

In [23]:
lemmas_f=m.lemmatize(lemmas)

In [24]:
print(lemmas_f)

['покупка', ' ', 'жилье', '; ', 'приобретение', ' ', 'автомобиль', '; ', 'дополнительный', ' ', 'образование', '; ', 'сыграть', ' ', 'свадьба', '; ', 'операция', ' ', 'с', ' ', 'жилье', '; ', 'образование', '; ', 'на', ' ', 'проведение', ' ', 'свадьба', '; ', 'покупка', ' ', 'жилье', ' ', 'для', ' ', 'семья', '; ', 'покупка', ' ', 'недвижимость', '; ', 'покупка', ' ', 'коммерческий', ' ', 'недвижимость', '; ', 'покупка', ' ', 'жилой', ' ', 'недвижимость', '; ', 'строительство', ' ', 'собственный', ' ', 'недвижимость', '; ', 'недвижимость', '; ', 'строительство', ' ', 'недвижимость', '; ', 'на', ' ', 'покупка', ' ', 'подержать', ' ', 'автомобиль', '; ', 'на', ' ', 'покупка', ' ', 'свой', ' ', 'автомобиль', '; ', 'операция', ' ', 'с', ' ', 'коммерческий', ' ', 'недвижимость', '; ', 'строительство', ' ', 'жилой', ' ', 'недвижимость', '; ', 'жилье', '; ', 'операция', ' ', 'со', ' ', 'свой', ' ', 'недвижимость', '; ', 'автомобиль', '; ', 'заниматься', ' ', 'образование', '; ', 'сделка', ' '

In [25]:
from collections import Counter
print(Counter(lemmas_f)) 

Counter({' ': 59, '; ': 37, 'покупка': 10, 'недвижимость': 10, 'автомобиль': 9, 'образование': 9, 'жилье': 7, 'с': 5, 'операция': 4, 'на': 4, 'свой': 4, 'свадьба': 3, 'строительство': 3, 'получение': 3, 'высокий': 3, 'дополнительный': 2, 'для': 2, 'коммерческий': 2, 'жилой': 2, 'подержать': 2, 'заниматься': 2, 'сделка': 2, 'приобретение': 1, 'сыграть': 1, 'проведение': 1, 'семья': 1, 'собственный': 1, 'со': 1, 'профильный': 1, 'сдача': 1, 'ремонт': 1, '\n': 1})


## Is there a relationship between having children and repaying a loan on time?

In [26]:
data.head()

,children,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,years_employed
0,1,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875,покупка жилья,23
1,1,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080,приобретение автомобиля,11
2,0,33,среднее,1,женат / замужем,0,M,сотрудник,0,145885,покупка жилья,15
3,3,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628,дополнительное образование,11
4,0,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616,сыграть свадьбу,932


In [27]:
children_pivot=data.pivot_table(index='children', columns='debt', values='purpose', aggfunc='count')

In [28]:
print(children_pivot)

debt            0       1
children                 
0         13028.0  1063.0
1          4410.0   445.0
2          1926.0   202.0
3           303.0    27.0
4            37.0     4.0
5             9.0     NaN


In [29]:
children_pivot['ratio']=children_pivot[1]/children_pivot[0]

In [30]:
print(children_pivot)

debt            0       1     ratio
children                           
0         13028.0  1063.0  0.081593
1          4410.0   445.0  0.100907
2          1926.0   202.0  0.104881
3           303.0    27.0  0.089109
4            37.0     4.0  0.108108
5             9.0     NaN       NaN


## Is there a relationship between marital status and loan repayment on time?

In [31]:
family_pivot=data.pivot_table(index='family_status', columns='debt', values='purpose', aggfunc='count')

In [32]:
print(family_pivot)

debt                       0    1
family_status                    
в разводе               1110   85
вдовец / вдова           896   63
гражданский брак        3763  388
женат / замужем        11408  931
не женат / не замужем   2536  274


In [33]:
family_pivot['ratio']=family_pivot[1]/family_pivot[0]

In [34]:
print(family_pivot)

debt                       0    1     ratio
family_status                              
в разводе               1110   85  0.076577
вдовец / вдова           896   63  0.070312
гражданский брак        3763  388  0.103109
женат / замужем        11408  931  0.081609
не женат / не замужем   2536  274  0.108044


## Is there a relationship between income level and loan repayment on time?

In [35]:
data.head()

,children,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,years_employed
0,1,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875,покупка жилья,23
1,1,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080,приобретение автомобиля,11
2,0,33,среднее,1,женат / замужем,0,M,сотрудник,0,145885,покупка жилья,15
3,3,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628,дополнительное образование,11
4,0,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616,сыграть свадьбу,932


In [36]:
data['total_income'].quantile([0.25,0.5,0.75])

0.25     89088.50
0.50    135781.00
0.75    195813.25
Name: total_income, dtype: float64

In [37]:
def income_type(total_income):
    if total_income<=89088.5:
        return 'Низкий'
    if total_income<=135781:
        return 'Средний'
    if total_income<=195813.25:
        return 'Высокий'
    return 'Сверхвысокий'
    

In [38]:
data['income_type']=data['total_income'].apply(income_type)

In [39]:
data.head()

,children,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,years_employed
0,1,42,высшее,0,женат / замужем,0,F,Сверхвысокий,0,253875,покупка жилья,23
1,1,36,среднее,1,женат / замужем,0,F,Средний,0,112080,приобретение автомобиля,11
2,0,33,среднее,1,женат / замужем,0,M,Высокий,0,145885,покупка жилья,15
3,3,32,среднее,1,женат / замужем,0,M,Сверхвысокий,0,267628,дополнительное образование,11
4,0,53,среднее,1,гражданский брак,1,F,Высокий,0,158616,сыграть свадьбу,932


In [40]:
income_pivot=data.pivot_table(index='income_type', columns='debt', values='purpose', aggfunc='count')

In [41]:
print(income_pivot)

debt             0    1
income_type            
Высокий       4882  481
Низкий        4945  419
Сверхвысокий  4981  383
Средний       4905  458


In [42]:
income_pivot['ratio']=income_pivot[1]/income_pivot[0]

In [43]:
print(income_pivot)

debt             0    1     ratio
income_type                      
Высокий       4882  481  0.098525
Низкий        4945  419  0.084732
Сверхвысокий  4981  383  0.076892
Средний       4905  458  0.093374


## How do different purposes of a loan affect its repayment on time?

In [44]:
def purpose_type(purpose):
    if 'образован' in purpose:
        return 'Образование'
    if 'авто' in purpose:
        return 'Автомобиль'
    if 'свадьб' in purpose:
        return 'Свадьба'
    return 'Недвижимость'

In [45]:
data['purpose_type']=data['purpose'].apply(purpose_type)

In [46]:
data.head()

,children,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,years_employed,purpose_type
0,1,42,высшее,0,женат / замужем,0,F,Сверхвысокий,0,253875,покупка жилья,23,Недвижимость
1,1,36,среднее,1,женат / замужем,0,F,Средний,0,112080,приобретение автомобиля,11,Автомобиль
2,0,33,среднее,1,женат / замужем,0,M,Высокий,0,145885,покупка жилья,15,Недвижимость
3,3,32,среднее,1,женат / замужем,0,M,Сверхвысокий,0,267628,дополнительное образование,11,Образование
4,0,53,среднее,1,гражданский брак,1,F,Высокий,0,158616,сыграть свадьбу,932,Свадьба


In [47]:
purpose_pivot=data.pivot_table(index='purpose_type', columns='debt', values='purpose', aggfunc='count')

In [48]:
print(purpose_pivot)

debt              0    1
purpose_type            
Автомобиль     3903  403
Недвижимость  10029  782
Образование    3643  370
Свадьба        2138  186


In [49]:
purpose_pivot['ratio']=purpose_pivot[1]/purpose_pivot[0]

In [50]:
print(purpose_pivot)

debt              0    1     ratio
purpose_type                      
Автомобиль     3903  403  0.103254
Недвижимость  10029  782  0.077974
Образование    3643  370  0.101565
Свадьба        2138  186  0.086997
